#  How to create a Person ReIdentification Application using Panorama SDK

**Goal of this Notebook** :

* Aid an Panorama developer prototype their application before creating the AWS Lambda for Panorama
* Using the built in wrapper application that **mimics** the Panorama sdk to get inference from the model

**What this Notebook accomplishes?** :
* Detect People in a selected video
* Using feature extraction to re-identify the person
* Use centroids to identify the direction of the movement 
* Use the direction to determine if the person entered or exited the building and display the entry/exit count on the video frames
* Draw bounding boxes around the person after re-identification



**CAUTION PLEASE READ BEFORE PROCEEDING** :

* The panoramasdk wrapper class used in this demo is not the original Panorama sdk that is on the device image
* The wrapper class does not reflect the capabilities of the original Panorama SDK on the device
* Its sole purpose is to give a developer a realistic idea of the structure and signature of the sdk on the device

In [18]:
# Video to use
video_to_use = "MOT16-03.mp4"

In [19]:
from __future__ import division
from __future__ import print_function

from IPython.display import clear_output, Markdown, display
import json

from gluoncv import model_zoo, data, utils
import mxnet as mx

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,20)


ModuleNotFoundError: No module named 'mxnet'

## Step 2: Modelling Approach

This step walks through using the Panorama SDK (wrapper) model to get inference
* Model: ssd_512_resnet50_v1_voc
* Dataset: These models are trained on PascalVOC datasets with 20 classes of objects
* arXiv: Application of Convolutional Neural Network for Image Classification on Pascal VOC Challenge 2012 dataset
* Model Input Size: 512 x 512
* Model Output: (1, 100, 1), (1,100,1), (1,100,4)
* Isolate people using class index 14 with a confidence higher than 30%
* Re-identify the person using torch reid

** A. Loading the model **

In [13]:
import os
import sys
path = os.path.abspath(os.path.join(os.path.dirname("panorama_sdk"), '../..'))
sys.path.insert(1, path + '/panorama_sdk')


import jupyter_utils

jupyter_utils.declare_globals({'mxnet_modelzoo_example': True, 
                               'custom_model': False, 'task':'object_detection', 'framework':'MXNET'})

import panoramasdk

print('Loading Model')
model = panoramasdk.model()
model.open('ssd_512_resnet50_v1_voc', 1)
print('Model Loaded')

ModuleNotFoundError: No module named 'jupyter_utils'

** B. Pre processing **

In [ ]:
import cv2

def letterbox_image(img, inp_dim):
    '''resize image with unchanged aspect ratio using padding'''
    img_w, img_h = img.shape[1], img.shape[0]
    w, h = inp_dim
    new_w = int(img_w * min(w/img_w, h/img_h))
    new_h = int(img_h * min(w/img_w, h/img_h))
    resized_image = cv2.resize(img, (new_w,new_h), interpolation = cv2.INTER_CUBIC)
    canvas = np.full((inp_dim[1], inp_dim[0], 3), 128)
    canvas[(h-new_h)//2:(h-new_h)//2 + new_h,(w-new_w)//2:(w-new_w)//2 + new_w,  :] = resized_image
    return canvas


def preprocess(img, h_size, w_size, letterbox=False):
    if letterbox:
        resized = letterbox_image(img, (w_size,h_size))
    else:
        resized = cv2.resize(img, (w_size,h_size), interpolation = cv2.INTER_CUBIC)
    mean = [0.485, 0.456, 0.406]  # RGB
    std = [0.229, 0.224, 0.225]  # RGB
    # converting array of ints to floats
    img = resized.astype(np.float32) / 255. 
    img_a = img[:, :, 0]
    img_b = img[:, :, 1]
    img_c = img[:, :, 2]
    # Extracting single channels from 3 channel image
    # The above code could also be replaced with cv2.split(img)
    # normalizing per channel data:
    img_a = (img_a - mean[0]) / std[0]
    img_b = (img_b - mean[1]) / std[1]
    img_c = (img_c - mean[2]) / std[2]
    # putting the 3 channels back together:
    x1 = [[[], [], []]]
    x1[0][0] = img_a
    x1[0][1] = img_b
    x1[0][2] = img_c
    x1 = np.asarray(x1)
    return x1

** C. Inference **

In [ ]:
import cv2
import numpy as np


## Panorama has a unique signature where we have to create empty arrays with the output dimensions before hand

# Create input and output arrays.
class_info = model.get_output(0)
prob_info = model.get_output(1)
rect_info = model.get_output(2)

class_array = np.empty(class_info.get_dims(), dtype=class_info.get_type())
prob_array = np.empty(prob_info.get_dims(), dtype=prob_info.get_type())
rect_array = np.empty(rect_info.get_dims(), dtype=rect_info.get_type())

person_image = cv2.imread('MOTS20-07.jpg')

# Pre Process Frame
x1 = preprocess(person_image, 512, 512, True)

# Do inference on the new frame.
model.batch(0, x1)
model.flush()

# Get the results.
resultBatchSet = model.get_result()

class_batch = resultBatchSet.get(0)
prob_batch = resultBatchSet.get(1)
rect_batch = resultBatchSet.get(2)

class_batch.get(0, class_array)
prob_batch.get(1, prob_array)
rect_batch.get(2, rect_array)

class_data = class_array
prob_data = prob_array
rect_data = rect_array


print('Class data shape is ', class_data.shape)
print('Confidence data shape is ', prob_data.shape)
print('Bounding Boxes data shape is ',rect_data.shape)

** D. Identify person with 30% or more confidence **

In [20]:
def get_persons(class_data, prob_data):
    # Returns an array the size of class data
    # Default false unless the identified object
    # is a person with probablity greater than 
    # the threshold
    person_indices = None
    number_of_people = 0
    if class_data is not None and len(class_data):
        person_indices = np.full(len(class_data), False)
        for x in range(len(class_data)):
            if int(class_data[x]) == 14 and prob_data[x] >= 0.30:
                person_indices[x] = True
                number_of_people += 1
    return person_indices

In [21]:
person_indices = get_persons(class_data[0], prob_data[0])
class_data = np.compress(person_indices, class_data[0], axis=0)
prob_data = np.compress(person_indices, prob_data[0], axis=0)
rect_data = np.compress(person_indices, rect_data[0], axis=0)

NameError: name 'class_data' is not defined

** E. Post processing utilities **

In [ ]:
def scale_coords(img1_shape, coords, img0_shape, ratio_pad=None):
    # Rescale coords (xyxy) from img1_shape to img0_shape
    if ratio_pad is None:  # calculate from img0_shape
        gain = min(img1_shape[0] / img0_shape[0], img1_shape[1] / img0_shape[1])  # gain  = old / new
        pad = (img1_shape[1] - img0_shape[1] * gain) / 2, (img1_shape[0] - img0_shape[0] * gain) / 2  # wh padding
    else:
        gain = ratio_pad[0][0]
        pad = ratio_pad[1]

    coords[:, [0, 2]] -= pad[0]  # x padding
    coords[:, [1, 3]] -= pad[1]  # y padding
    coords[:, :4] /= gain
    coords = clip_coords(coords, img0_shape)
    return coords


def clip_coords(boxes, img_shape):
    # Clip bounding xyxy bounding boxes to image shape (height, width)
    boxes[:, 0] = np.clip(boxes[:, 0], 0, img_shape[1])  # x1
    boxes[:, 1] = np.clip(boxes[:, 1], 0, img_shape[0])  # y1
    boxes[:, 2] = np.clip(boxes[:, 2], 0, img_shape[1])  # x2
    boxes[:, 3] = np.clip(boxes[:, 3], 0, img_shape[0])  # y2
    return boxes

def xyxy_to_xywh(boxes_xyxy):
    boxes_xywh = boxes_xyxy.copy()
    boxes_xywh[:, 0] = (boxes_xyxy[:, 0] + boxes_xyxy[:, 2]) / 2.
    boxes_xywh[:, 1] = (boxes_xyxy[:, 1] + boxes_xyxy[:, 3]) / 2.
    boxes_xywh[:, 2] = boxes_xyxy[:, 2] - boxes_xyxy[:, 0]
    boxes_xywh[:, 3] = boxes_xyxy[:, 3] - boxes_xyxy[:, 1]
    for index in range(len(boxes_xywh)):
        log(f'X:{boxes_xywh[index][0]} Y: {boxes_xywh[index][1]} W: {boxes_xywh[index][2]} H: {boxes_xywh[index][3]}')
    return boxes_xywh

** F. Load the re-identification model **

In [ ]:
import jupyter_utils

jupyter_utils.declare_globals({'mxnet_modelzoo_example': False, 
                               'custom_model': True, 'task':'PERSON_REID', 'framework':'PYTORCH'})

import panoramasdk
from importlib import reload  

panoramasdk = reload(panoramasdk)

print('Loading Model')
model = panoramasdk.model()
# PLEASE REFER TO Torch-reid.ipynb TO CREATE THIS MODEL
model.open('reid_model_v1', 1)
print('Model Loaded')

** G. Re-identification using deep sort **

In [ ]:
import os
import sys

path = os.path.abspath(os.path.join(os.path.dirname("Lambda"), './..'))
sys.path.insert(1, path + '/Lambda')

In [ ]:
from deep_sort import build_tracker

cfg = {}
cfg["INPUT_SIZE_H"] = 256
cfg["INPUT_SIZE_W"] = 128
cfg["MAX_DIST"] = 0.5
cfg["MIN_CONFIDENCE"] = 0.3
cfg["NMS_MAX_OVERLAP"] = 0.8
cfg["MAX_IOU_DISTANCE"] = 0.7
cfg["MAX_AGE"] = 70
cfg["N_INIT"] = 5
cfg["NN_BUDGET"] = 20
deepsort = build_tracker(tracking_model, cfg, True)
